<a href="https://colab.research.google.com/github/OpenAIMP/XINN/blob/main/taxi_RL_using_Rlib_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ray[rllib]

     |████████████████████████████████| 49.6 MB 6.8 kB/s 
     |████████████████████████████████| 72 kB 522 kB/s 
     |████████████████████████████████| 124 kB 87.5 MB/s 
     |████████████████████████████████| 1.8 MB 34.3 MB/s 


In [ ]:
pip install gym

In [ ]:
pip install tensorflow

In [ ]:
import ray
import ray.rllib.agents.ppo as ppo

ray.shutdown()
ray.init(ignore_reinit_error=True)


In [ ]:
import shutil
import os


CHECKPOINT_ROOT = "tmp/ppo/taxi"
shutil.rmtree(CHECKPOINT_ROOT, ignore_errors=True, onerror=None)

ray_results = os.getenv("HOME") + "/ray_results/"
shutil.rmtree(ray_results, ignore_errors=True, onerror=None)

In [ ]:
SELECT_ENV = "Taxi-v3"

config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"

agent = ppo.PPOTrainer(config, env=SELECT_ENV)

2021-09-28 09:13:22,918	INFO trainer.py:714 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-09-28 09:13:22,919	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2021-09-28 09:13:22,920	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-09-28 09:13:30,194	WARNING util.py:55 -- Install gputil for GPU system monitoring.


In [ ]:
N_ITER = 32
s = "{:3d} reward {:6.2f}/{:6.2f}/{:6.2f} len {:6.2f} saved {}"

for n in range(N_ITER):
  result = agent.train()
  file_name = agent.save(CHECKPOINT_ROOT)

  print(s.format(
    n + 1,
    result["episode_reward_min"],
    result["episode_reward_mean"],
    result["episode_reward_max"],
    result["episode_len_mean"],
    file_name
   ))

  1 reward -209.00/-45.73/ 15.00 len  61.48 saved tmp/ppo/taxi/checkpoint_000126/checkpoint-126
  2 reward -209.00/-50.69/ 13.00 len  65.69 saved tmp/ppo/taxi/checkpoint_000127/checkpoint-127
  3 reward -200.00/-63.78/ 15.00 len  77.40 saved tmp/ppo/taxi/checkpoint_000128/checkpoint-128
  4 reward -209.00/-59.63/ 15.00 len  73.85 saved tmp/ppo/taxi/checkpoint_000129/checkpoint-129
  5 reward -218.00/-59.37/ 14.00 len  73.41 saved tmp/ppo/taxi/checkpoint_000130/checkpoint-130
  6 reward -209.00/-44.66/ 14.00 len  60.35 saved tmp/ppo/taxi/checkpoint_000131/checkpoint-131
  7 reward -200.00/-51.26/ 14.00 len  66.50 saved tmp/ppo/taxi/checkpoint_000132/checkpoint-132
  8 reward -209.00/-67.93/ 12.00 len  81.31 saved tmp/ppo/taxi/checkpoint_000133/checkpoint-133
  9 reward -209.00/-82.85/ 14.00 len  94.40 saved tmp/ppo/taxi/checkpoint_000134/checkpoint-134
 10 reward -209.00/-71.45/ 14.00 len  83.96 saved tmp/ppo/taxi/checkpoint_000135/checkpoint-135
 11 reward -209.00/-77.20/ 12.00 len  89

In [ ]:
policy = agent.get_policy()
model = policy.model
print(model.base_model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
observations (InputLayer)       [(None, 500)]        0                                            
__________________________________________________________________________________________________
fc_1 (Dense)                    (None, 256)          128256      observations[0][0]               
__________________________________________________________________________________________________
fc_value_1 (Dense)              (None, 256)          128256      observations[0][0]               
__________________________________________________________________________________________________
fc_2 (Dense)                    (None, 256)          65792       fc_1[0][0]                       
______________________________________________________________________________________________